In [1]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

from utils import *
from equations import *

In [2]:
epochADAM = 400000
epochLBFGS = 0
N_u=4000
N_H=4000
N_f=9000
N_C=None
N_mu=4000
seed=1234
log_frequency=10000
history_frequency=10
NLayers=6
NNeurons=40
noiseLevel=[]
weights = [5,3,5,10,16,16]

In [3]:
hp = {}
# Data size on the solution u
hp["N_u"] = N_u
hp["N_s"] = N_u
hp["N_H"] = N_H
hp["N_C"] = N_C
hp["N_mu"] = N_mu
# Collocation points size, where we’ll check for f = 0
hp["N_f"] = N_f
# DeepNN topology (2-sized input [x,y], NLayers hidden layer of NNeurons-width, 1-sized output [u,v]
hp["layers"] = [2]+[NNeurons]*NLayers+[2]
# DeepNN topology (1-sized input [x,y], NLayers hidden layer of NNeurons-width, 2-sized output [h, H]
hp["h_layers"] = [2]+[NNeurons]*NLayers+[2]
# DeepNN topology (1-sized input [x,y], NLayers hidden layer of NNeurons-width, 1-sized output [C]
hp["C_layers"] = [2]+[NNeurons]*NLayers+[1]
# DeepNN topology (1-sized input [x,y], NLayers hidden layer of NNeurons-width, 1-sized output [C]
hp["mu_layers"] = [2]+[NNeurons]*NLayers+[1]
# DeepNN topology (1-sized input [x], NLayers hidden layer of NNeurons-width, 4-sized input [u,v,C], 
#   1-sized output [taub]
hp["friction_layers"] = [4]+[NNeurons]*NLayers+[1]
# Setting up the TF SGD-based optimizer (set tf_epochs=0 to cancel it)
hp["tf_epochs"] = epochADAM
hp["tf_lr"] = 0.001
hp["tf_b1"] = 0.99
hp["tf_eps"] = 1e-1
# Setting up the quasi-newton LBGFS optimizer (set nt_epochs=0 to cancel it)
hp["nt_epochs"] = epochLBFGS
hp["log_frequency"] = log_frequency
# Record the history
hp["save_history"] = True
hp["history_frequency"] = history_frequency

# path for loading data and saving models
repoPath = "./"
appDataPath = os.path.join(repoPath, "matlab_SSA", "DATA")
path = os.path.join(appDataPath, "Helheim_Big_PINN_fastflow_Rignot_SMW_JoughinComposite.mat")

modelPath = './Models/SSA2D_4NN_3NN_6x20_weights5_3_5_8_14_8_20231124_143323'

reloadModel = True
weights = [5, 3, 5, 10, 16]
loss_weights = [10**(-w) for w in weights]


In [4]:
x, y, Exact_vx, Exact_vy, X_star, u_star, X_u_train, u_train, X_f, X_bc, u_bc, X_cf, n_cf, xub, xlb, uub, ulb, mu = prep_2D_data_withmu(path, N_f=hp["N_f"], N_u=hp["N_u"], N_s=hp["N_s"], N_H=hp["N_H"], N_C=hp["N_C"], N_mu=hp["N_mu"])

In [5]:
X_u_train['H'].shape

(500, 2)

In [6]:
logger = Logger(hp)
pinn = SSA_4NN(hp, logger, X_f,
        X_cf, n_cf,
        xub, xlb, uub, ulb,
        modelPath, reloadModel,
        loss_weights=loss_weights)

Hyperparameters:
{
  "N_u": 4000,
  "N_s": 4000,
  "N_H": 500,
  "N_C": null,
  "N_mu": 4000,
  "N_f": 9000,
  "layers": [
    2,
    40,
    40,
    40,
    40,
    40,
    40,
    2
  ],
  "h_layers": [
    2,
    40,
    40,
    40,
    40,
    40,
    40,
    2
  ],
  "C_layers": [
    2,
    40,
    40,
    40,
    40,
    40,
    40,
    1
  ],
  "mu_layers": [
    2,
    40,
    40,
    40,
    40,
    40,
    40,
    1
  ],
  "friction_layers": [
    4,
    40,
    40,
    40,
    40,
    40,
    40,
    1
  ],
  "tf_epochs": 400000,
  "tf_lr": 0.001,
  "tf_b1": 0.99,
  "tf_eps": 0.1,
  "nt_epochs": 0,
  "log_frequency": 10000,
  "save_history": true,
  "history_frequency": 10
}

TensorFlow version: 2.4.1
Eager execution: True
GPU-accerelated: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


OSError: SavedModel file does not exist at: ./Models/SSA2D_FlightTrackH_3NN_6x20_weights5_3_5_8_14_20231126_010143/mu_model//{saved_model.pbtxt|saved_model.pb}

In [ ]:
pinn.model.layers[-1].weights

In [ ]:
pinn.h_model.layers[-1].weights

In [ ]:
    yts = 3600*24*365
    X, Y = np.meshgrid(np.linspace(xlb[0],xub[0],200), np.linspace(xlb[1],xub[1], 200))
    # obs
    ux = yts*griddata(X_star, u_star[:,0].flatten(), (X, Y), method='cubic')
    uy = yts*griddata(X_star, u_star[:,1].flatten(), (X, Y), method='cubic')
    h_obs = griddata(X_star, u_star[:,2].flatten(), (X, Y), method='cubic')
    H_obs = griddata(X_star, u_star[:,3].flatten(), (X, Y), method='cubic')
    C_obs = griddata(X_star, u_star[:,4].flatten(), (X, Y), method='cubic')
    mu_obs = griddata(X_star, u_star[:,5].flatten(), (X, Y), method='cubic')

    # predicted solution
    u_pred, v_pred, h, H, C_pred, mu = pinn.predict(X_star)
    u_nn = yts*griddata(X_star, u_pred[:,0].flatten(), (X, Y), method='cubic')
    v_nn = yts*griddata(X_star, v_pred[:,0].flatten(), (X, Y), method='cubic')
    C_nn = griddata(X_star, C_pred[:,0], (X, Y), method='cubic')
    h_nn = griddata(X_star, h[:,0], (X, Y), method='cubic')
    H_nn = griddata(X_star, H[:,0], (X, Y), method='cubic')
    mu_nn = griddata(X_star, mu[:,0], (X, Y), method='cubic')

    H_interp = griddata(X_u_train['H'], u_train['H'].flatten(), (X, Y), method='cubic')
    C_interp = griddata(X_u_train['C'], u_train['C'].flatten(), (X, Y), method='cubic')
    mu_interp = griddata(X_u_train['mu'], u_train['mu'].flatten(), (X, Y), method='cubic')
    # residual
    f1, f2 = pinn.f_model()
    F1 = griddata(X_f, f1[:,0], (X, Y), method='cubic')
    F2 = griddata(X_f, f2[:,0], (X, Y), method='cubic')

In [ ]:
    vranges={'vel obs': [0,8e3], 'vel': [0,8e3], 'vel-vel obs':[-1e3,1e3], 
             'H':[0,1500], 'H obs':[0,1500],'H interp':[0,1500], 
             'H-H obs':[-0.25e3,0.25e3],'H-H interp':[-0.25e3,0.25e3], 
             'C':[0,8e3], 'C obs':[0,8e3],'C-C obs':[-2e3,2e3],
             'mu':[0.5e8,1.8e8], 'mu obs':[0.5e8,1.8e8],'mu-mu obs':[-1e8, 1e8],
             's-s obs':[-100,100], 's':[-500,2000], 's obs':[-500,2000]
          }
    ###########################
    plotData = {}
    plotData['vel obs'] = np.sqrt(ux**2+uy**2)
    plotData['s obs'] = h_obs
    plotData['C obs'] = C_obs
    plotData['mu obs'] = mu_obs
    plotData['H obs'] = H_obs
    ###########################
    vel = np.sqrt(u_nn**2+v_nn**2)
    plotData['vel'] = vel
    plotData['s'] = h_nn
    plotData['C'] = np.abs(C_nn)
    plotData['mu'] = np.abs(mu_nn)
    plotData['H'] = np.abs(H_nn)
    ###########################
    plotData['vel-vel obs'] =  plotData['vel'] -  plotData['vel obs'] 
    plotData['s-s obs'] = h_nn - h_obs
    plotData['C-C obs'] = plotData['C'] - C_obs
    plotData['mu-mu obs'] = plotData['mu'] - C_obs
    plotData['H-H obs'] = plotData['H'] - H_obs
    ###########################
    plotData['H interp'] = H_interp
    plotData['H-H interp'] = (H_obs - H_interp)
    plotData['C interp'] = C_interp
    plotData['Training data'] = 0*H_obs

#     flagthin = (vel<2000)
#     plotData['H-H interp'][flagthin] = 0
#     plotData['H-H obs'][flagthin] = 0
    
    fig, axs = plt.subplots(4, 5 , figsize=(16,12))

    for ax,name in zip(axs.ravel(), plotData.keys()):
        vr = vranges.setdefault(name, [None, None])
        im = ax.imshow(plotData[name], interpolation='nearest', cmap='rainbow',
                extent=[X.min(), X.max(), Y.min(), Y.max()],
                vmin=vr[0], vmax=vr[1],
                origin='lower', aspect='auto')
        ax.set_title(name)
        fig.colorbar(im, ax=ax, shrink=1)
        if name =='Training data':
            ax.scatter(X_u_train['H'][:,0:1],X_u_train['H'][:,1:2],1,'black')
            ax.scatter(X_u_train['C'][:,0:1],X_u_train['C'][:,1:2],1,'red')
            ax.set_xlim(X.min(), X.max())
            ax.set_ylim(Y.min(), Y.max())